In [2]:
import geopandas as gpd
import matplotlib.pyplot as plt
import contextily as ctx
import requests
from shapely.geometry import Point, Polygon, LineString, MultiLineString
import pandas as pd
import json
import geojson
import re


In [16]:
def extract_year(label_dict):
    if not label_dict or '_label_' not in label_dict:
        return None
        
    text = label_dict['_label_']
    year_match = re.search(r'Rok budowy/remontu: (.*?)(?:\n|$)', text)
    
    if not year_match:
        return None
        
    year_value = year_match.group(1)
    
    # Handle 'brak danych' case
    if year_value == 'brak danych':
        return None
        
    # Try to extract number if exists
    if year_value.isdigit():
        return int(year_value)
        
    return None

In [ ]:

def process_message_json(json_file):
    # Read JSON array
    with open(json_file, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    i = 0
    lines = []
    ids = []
    years = []
    for item in data:
        # i += 1
        # if i > 500:
        #     break


        _id = item['_id']
        # if _id != 'ABeSwIAABAAP8lcAAC':
        #     continue
        geometry = item['geometry']
        properties = item['properties']
        year = extract_year(properties)
        if not year:
            continue
        if geometry['type'] != 'LineString':
            continue

        coords1 = []
        coords2 = []
        for index, coord in enumerate(geometry['coordinates']):
            if index % 2 == 0:
                coords1.append(coord)
            else:
                coords2.append(coord)
        
        coords = list(zip(coords1, coords2))
        lines.append(LineString(coords))
        ids.append(_id)
        years.append(year)

    gdf = gpd.GeoDataFrame({'id': ids, 'year': years}, geometry=lines, crs='EPSG:2178')
    return gdf



In [79]:
gdf = process_message_json('message.json')

# 4326
# 3857
gdf = gdf.to_crs(epsg=3857)
display(gdf.head())
print(gdf.crs)


,id,year,geometry
0,ABeSwIAABAAP8hhAAA,NaN,"LINESTRING (2356999.525 6873784.613, 2356945.7..."
1,ABeSwIAABAAP8o3AAA,NaN,"LINESTRING (2359648.281 6872499.633, 2359642.2..."
2,ABeSwIAABAAP8NoAAA,NaN,"LINESTRING (2359583.981 6872294.597, 2359565.6..."
3,ABeSwIAABAAP8mvAAC,NaN,"LINESTRING (2360254.521 6872491.211, 2360245.0..."
4,ABeSwIAABAAP8O1AAE,NaN,"LINESTRING (2359879.285 6872468.419, 2359873.8..."


EPSG:3857


In [ ]:

# Create figure
fig, ax = plt.subplots(figsize=(15, 15))

# Plot routes colored by year
gdf.plot( ax=ax,
    linewidth=1.5,
    color='blue',
    label='Bike Routes')

# # Add basemap
ctx.add_basemap(ax, source=ctx.providers.OpenStreetMap.Mapnik) 

# # Customize plot
# plt.title('Warsaw Bike Routes by Year', pad=20, size=16)
# plt.axis('off')

In [83]:
import matplotlib.animation as animation
from matplotlib.animation import FuncAnimation

def animate(frame_year):
    ax.clear()
    
    # Plot data up to current year
    mask = gdf['year'] <= frame_year
    gdf[mask].plot(
        ax=ax,
        color='blue',
        linewidth=1.5,
        alpha=0.7
    )
    
    # Add basemap
    ctx.add_basemap(ax, source=ctx.providers.OpenStreetMap.Mapnik)
    
    # Add title with year
    ax.set_title(f'Warsaw Bike Routes Development: {frame_year}')
    # ax.axis('off')

# Setup figure
fig, ax = plt.subplots(figsize=(15, 15))

# Get unique years for animation
years = sorted(gdf['year'].unique())

# Create animation
anim = FuncAnimation(
    fig, 
    animate,
    frames=years,
    interval=1000,  # 1 second between frames
    repeat=True
)

# Save animation
anim.save('bike_routes_timelapse.gif', writer='pillow')
plt.close()

C:\Users\Adrian\AppData\Local\Temp\ipykernel_26800\1279728287.py:9: UserWarning: The GeoDataFrame you are attempting to plot is empty. Nothing has been displayed.
  gdf[mask].plot(
d:\GitRepos\spatial-data-analysis\project\.venv\Lib\site-packages\contextily\tile.py:645: UserWarning: The inferred zoom level of 27 is not valid for the current tile provider (valid zooms: 0 - 19).
  warnings.warn(msg)
C:\Users\Adrian\AppData\Local\Temp\ipykernel_26800\1279728287.py:9: UserWarning: The GeoDataFrame you are attempting to plot is empty. Nothing has been displayed.
  gdf[mask].plot(
d:\GitRepos\spatial-data-analysis\project\.venv\Lib\site-packages\contextily\tile.py:645: UserWarning: The inferred zoom level of 27 is not valid for the current tile provider (valid zooms: 0 - 19).
  warnings.warn(msg)


In [88]:
import matplotlib.animation as animation
from matplotlib.animation import FuncAnimation

# Define Warsaw center and bounds (in EPSG:3857)
WARSAW_CENTER = (2338552.675489827, 6841916.603611625)  # lon, lat
MARGIN = 30000  # degrees

def set_map_bounds(ax):
    # Set fixed bounds around Warsaw
    ax.set_xlim([WARSAW_CENTER[0] - MARGIN, WARSAW_CENTER[0] + MARGIN])
    ax.set_ylim([WARSAW_CENTER[1] - MARGIN, WARSAW_CENTER[1] + MARGIN])

def animate(frame_year):
    ax.clear()
    
    # Plot data up to current year
    mask = gdf['year'] <= frame_year
    gdf[mask].plot(
        ax=ax,
        color='blue',
        linewidth=1.5,
        alpha=0.7
    )
    
    # Set consistent bounds
    set_map_bounds(ax)
    
    # Add basemap with fixed zoom
    ctx.add_basemap(ax, source=ctx.providers.OpenStreetMap.Mapnik)
    
    ax.set_title(f'Warsaw Bike Routes Development: {frame_year}')
    # ax.axis('off')

# Setup figure
fig, ax = plt.subplots(figsize=(15, 15))
years = sorted(gdf['year'].unique())

# Create animation
anim = FuncAnimation(
    fig, 
    animate,
    frames=years,
    interval=1000,
    repeat=True
)

anim.save('bike_routes_timelapse.gif', writer='pillow')
plt.close()

C:\Users\Adrian\AppData\Local\Temp\ipykernel_26800\2955976333.py:18: UserWarning: The GeoDataFrame you are attempting to plot is empty. Nothing has been displayed.
  gdf[mask].plot(
C:\Users\Adrian\AppData\Local\Temp\ipykernel_26800\2955976333.py:18: UserWarning: The GeoDataFrame you are attempting to plot is empty. Nothing has been displayed.
  gdf[mask].plot(
